In [1]:
import pandas as pd
import re
import os

import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
quality_path = '../refcoco_api/results/'
mcn_path =  '../data/mcn_results/'

In [3]:
def index_names(name):

    patterns = [
        (r'.*beam_l-na_r-na', 'beam'),
        (r'.*greedy_l-na_r-na', 'greedy'),
        (r'.*predfuse_es_l-0-3_r-na', 'es03'),
        (r'.*predfuse_es_l-0-5_r-na', 'es05'),
        (r'.*predfuse_es_l-0-7_r-na', 'es07'),
        (r'.*rsa_l-na_r-0-5', 'rsa05'),
        (r'.*rsa_l-na_r-1-0', 'rsa10'),
        (r'.*rsa_l-na_r-5-0', 'rsa50'),
        (r'.*annsample', 'ann')
    ]
    
    extra_patterns = [
        (r'.*predfuse_es_l-0-1_r-na', 'es01'),
        (r'.*predfuse_es_l-0-2_r-na', 'es02'),
        (r'.*predfuse_es_l-0-4_r-na', 'es04'),
        (r'.*predfuse_es_l-0-6_r-na', 'es06'),
        (r'.*predfuse_es_l-0-8_r-na', 'es08'),
        (r'.*predfuse_es_l-0-9_r-na', 'es09'),
    ]
    
    patterns += extra_patterns
    
    for pattern, replace in patterns:
        name = re.sub(pattern, replace, name)
    
    return name


def reformat_index(df, ordered_index=['greedy', 'beam', 'es07', 'es05', 'es03', 'rsa05', 'rsa10', 'rsa50', 'ann']):

    df.index = df.index.map(index_names)
    df.index = pd.Categorical(df.index, 
                              categories=ordered_index,
                              ordered=True)
    return df


def get_split(x):
    return x.split('_')[1]

# quality

In [4]:
quality_files = [os.path.join(quality_path, file) for file in os.listdir(quality_path)]

In [5]:
refcoco_file = os.path.join(quality_path, 'refcoco_quality_results.csv')
refcocop_file = os.path.join(quality_path, 'refcoco+_quality_results.csv')
refcocog_file = os.path.join(quality_path, 'refcocog_quality_results.csv')

In [6]:
ordered_index = ['greedy', 'beam', 'es07', 'es05', 'es03', 'rsa05', 'rsa10', 'rsa50', 'ann']
selected_columns = [
    'Bleu_1', 
#    'Bleu_2', 
    'CIDEr'
]
_round = {'Bleu_1': 10, 'CIDEr': 20}

In [7]:
refcoco_quality = pd.read_csv(refcoco_file, index_col=0)
refcoco_quality['split'] = refcoco_quality.index.map(get_split)
refcoco_quality = reformat_index(refcoco_quality)
refcoco_quality['Bleu_1'] = refcoco_quality['Bleu_1'] * 100
refcoco_quality = refcoco_quality.round(_round)

refcoco_quality_testA = refcoco_quality.loc[refcoco_quality.split == 'testA'].sort_index()[selected_columns]
refcoco_quality_testA.columns = pd.MultiIndex.from_product([["testA"], refcoco_quality_testA.columns])

refcoco_quality_testB = refcoco_quality.loc[refcoco_quality.split == 'testB'].sort_index()[selected_columns]
refcoco_quality_testB.columns = pd.MultiIndex.from_product([["testB"], refcoco_quality_testB.columns])

refcocop_quality = pd.read_csv(refcocop_file, index_col=0)
refcocop_quality['split'] = refcocop_quality.index.map(get_split)
refcocop_quality = reformat_index(refcocop_quality)
refcocop_quality['Bleu_1'] = refcocop_quality['Bleu_1'] * 100
refcocop_quality = refcocop_quality.round(_round)

refcocop_quality_testA = refcocop_quality.loc[refcocop_quality.split == 'testA'].sort_index()[selected_columns]
refcocop_quality_testA.columns = pd.MultiIndex.from_product([["testA+"], refcocop_quality_testA.columns])

refcocop_quality_testB = refcocop_quality.loc[refcocop_quality.split == 'testB'].sort_index()[selected_columns]
refcocop_quality_testB.columns = pd.MultiIndex.from_product([["testB+"], refcocop_quality_testB.columns])

refcocog_quality = pd.read_csv(refcocog_file, index_col=0)
refcocog_quality['split'] = refcocog_quality.index.map(get_split)
refcocog_quality = reformat_index(refcocog_quality)
refcocog_quality['Bleu_1'] = refcocog_quality['Bleu_1'] * 100
refcocog_quality = refcocog_quality.round(_round)

refcocog_quality_test = refcocog_quality.loc[refcocog_quality.split == 'test'].sort_index()[selected_columns]
refcocog_quality_test.columns = pd.MultiIndex.from_product([["testg"], refcocog_quality_test.columns])

In [8]:
combined_quality_table = pd.concat([refcoco_quality_testA, refcoco_quality_testB, refcocop_quality_testA, refcocop_quality_testB, refcocog_quality_test], axis=1)
print(combined_quality_table.to_latex())

\begin{tabular}{lrrrrrrrrrr}
\toprule
{} & \multicolumn{2}{l}{testA} & \multicolumn{2}{l}{testB} & \multicolumn{2}{l}{testA+} & \multicolumn{2}{l}{testB+} & \multicolumn{2}{l}{testg} \\
{} &      Bleu\_1 &     CIDEr &      Bleu\_1 &     CIDEr &      Bleu\_1 &     CIDEr &      Bleu\_1 &     CIDEr &      Bleu\_1 &     CIDEr \\
\midrule
greedy &   53.247874 &  0.792400 &   56.152687 &  1.268892 &   45.609185 &  0.678206 &   33.932929 &  0.636696 &   44.985229 &  0.705057 \\
beam   &   52.823791 &  0.806644 &   55.438707 &  1.312195 &   40.432405 &  0.663705 &   32.527953 &  0.748805 &   45.003220 &  0.786220 \\
es07   &   52.864722 &  0.799691 &   55.705072 &  1.240842 &   40.269092 &  0.670867 &   30.882333 &  0.708353 &   44.086456 &  0.744497 \\
es05   &   49.814756 &  0.733767 &   53.261126 &  1.114785 &   37.113972 &  0.602526 &   27.437358 &  0.605249 &   40.428734 &  0.629020 \\
es03   &   35.778148 &  0.527885 &   44.535568 &  0.807336 &   23.102764 &  0.374305 &   21.034349 &  0.

# MCN

In [9]:
mcn_files = [os.path.join(mcn_path, file) for file in os.listdir(mcn_path)]

In [10]:
seg_pattern = r'iou: (\d\.\d+)'
det_pattern = r'Acc@\.5: (\d\.\d+)'
ie_pattern = r'IE score : (\d\.\d+)'

In [11]:
refcoco_files = [f for f in mcn_files if '_refcoco_' in f]
refcocop_files = [f for f in mcn_files if '_refcoco+_' in f]
refcocog_files = [f for f in mcn_files if '_refcocog_' in f]

In [12]:
def mcn2df(list_of_files): 
    mcn_df = pd.DataFrame()

    for file in sorted(list_of_files):
        with open(file) as f:
            data = f.read()

        seg_result = float(re.search(seg_pattern, data).group(1))
        det_result = float(re.search(det_pattern, data).group(1))
        ie_result = float(re.search(ie_pattern, data).group(1))

        name = os.path.split(file)[-1].replace('_cleaned_filtered.txt', '')

        mcn_df = mcn_df.append(pd.Series(
            {'segmentation': seg_result*100, 'detection': det_result*100, 'IE': ie_result},
            name=name
        ))
        
    def get_split(x):
        return x.split('_')[2]
        
    mcn_df['split'] = mcn_df.index.map(get_split)

    return mcn_df

In [13]:
refcoco_mcn = mcn2df(refcoco_files).round(1)
refcoco_mcn = reformat_index(refcoco_mcn)

refcocop_mcn = mcn2df(refcocop_files).round(1)
refcocop_mcn = reformat_index(refcocop_mcn)

refcocog_mcn = mcn2df(refcocog_files).round(1)
refcocog_mcn = reformat_index(refcocog_mcn)

In [14]:
mcn_selected_columns = [
    'detection', 
    #'segmentation'
]

_round = {'detection': 2, 'segmentation': 2}

In [15]:
refcoco_mcn_testA = refcoco_mcn.loc[refcoco_mcn.split == 'testA'].sort_index()[mcn_selected_columns].round(_round)
refcoco_mcn_testA.columns = pd.MultiIndex.from_product([["testA"], refcoco_mcn_testA.columns])

refcoco_mcn_testB = refcoco_mcn.loc[refcoco_mcn.split == 'testB'].sort_index()[mcn_selected_columns].round(_round)
refcoco_mcn_testB.columns = pd.MultiIndex.from_product([["testB"], refcoco_mcn_testB.columns])

refcocop_mcn_testA = refcocop_mcn.loc[refcocop_mcn.split == 'testA'].sort_index()[mcn_selected_columns].round(_round)
refcocop_mcn_testA.columns = pd.MultiIndex.from_product([["testA+"], refcocop_mcn_testA.columns])

refcocop_mcn_testB = refcocop_mcn.loc[refcocop_mcn.split == 'testB'].sort_index()[mcn_selected_columns].round(_round)
refcocop_mcn_testB.columns = pd.MultiIndex.from_product([["testB+"], refcocop_mcn_testB.columns])

refcocog_mcn_test = refcocog_mcn.loc[refcocog_mcn.split == 'test'].sort_index()[mcn_selected_columns].round(_round)
refcocog_mcn_test.columns = pd.MultiIndex.from_product([["testg"], refcocog_mcn_test.columns])

In [16]:
combined_mcn_table = pd.concat([refcoco_mcn_testA, refcoco_mcn_testB, refcocop_mcn_testA, refcocop_mcn_testB, refcocog_mcn_test], axis=1)
combined_mcn_table

In [18]:
refcoco_testA = pd.concat([refcoco_quality_testA, refcoco_mcn_testA], axis=1)
refcoco_testB = pd.concat([refcoco_quality_testB, refcoco_mcn_testB], axis=1)
refcocop_testA = pd.concat([refcocop_quality_testA, refcocop_mcn_testA], axis=1)
refcocop_testB = pd.concat([refcocop_quality_testB, refcocop_mcn_testB], axis=1)
refcocog_test = pd.concat([refcocog_quality_test, refcocog_mcn_test], axis=1)

In [19]:
combined_quality_mcn = pd.concat([refcoco_testA,refcoco_testB,refcocop_testA,refcocop_testB,refcocog_test], axis=1)
combined_quality_mcn

In [48]:
combined_quality_mcn.round(2).drop('ann').style.highlight_max(color = 'lightgreen', axis = 0, )